In [31]:
import tensorflow as tf
import keras
from keras.layers import LSTM, Dense, Dropout, Input
from keras.callbacks import TensorBoard
import os
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam

In [32]:
path = "gesture_data"

data = []
labels = []

for file_name in os.listdir(path):
    if file_name .endswith(".csv"):
        label = file_name.split("_")[0]
        file_path = os.path.join(path, file_name)

        df = pd.read_csv(file_path)
        data.append(df.values)
        labels.append(label)

In [33]:
np.array(data).shape, np.array(labels).shape

((88, 30, 63), (88,))

In [34]:
X = np.array(data)
y = np.array(labels)

In [35]:
label_encoder = LabelEncoder()
y_encoded = to_categorical(label_encoder.fit_transform(labels)) 

In [36]:
model = keras.Sequential([
    Input(shape=(30,63)),
    LSTM(64, return_sequences=True, activation='relu'),
    LSTM(32, return_sequences=False, activation='relu'),
    Dense(32, activation='relu'),
    Dense(6, activation='softmax')
])

In [37]:
adam = Adam(learning_rate=0.0001)
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [38]:
from keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True,)
history = model.fit(
    X,y_encoded,
    epochs=100,
    validation_split=0.2,
    batch_size=16,
    callbacks=callback
)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 703ms/step - accuracy: 0.1518 - loss: 1.7914 - val_accuracy: 0.2778 - val_loss: 1.7809
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1874 - loss: 1.7851 - val_accuracy: 0.2778 - val_loss: 1.7785
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2658 - loss: 1.7821 - val_accuracy: 0.2778 - val_loss: 1.7767
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3023 - loss: 1.7765 - val_accuracy: 0.2778 - val_loss: 1.7747
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3092 - loss: 1.7767 - val_accuracy: 0.2778 - val_loss: 1.7731
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3179 - loss: 1.7727 - val_accuracy: 0.2778 - val_loss: 1.7713
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2997 - loss: 1.7705 - val_accuracy: 0.2778 - val_loss: 1.7695
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2693 - loss: 1.7707 - val_accuracy: 0.2778 - val_loss

In [39]:
model.save('gesture_classifier.h5')

In [40]:
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Class Mapping:")
for class_name, encoded_value in class_mapping.items():
    print(f"{class_name}: {encoded_value}")

Class Mapping:
faster: 0
like: 1
next: 2
previous: 3
resume: 4
stop: 5
